# SBIRT GRPA Coding

Goverment Perfromance and Results Act (GPRA) coding for intake data of recovery specialist for the Institure for Prevention and Recovery (IFPR) at RWJ Baranabas

In [1]:
#importing pandas
import pandas as pd
import numpy as np
from datetime import datetime as dt
import string as str

In [2]:
# importing data and parsing datetime columns
df = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\GPRA\SBIRT_GPRA_Coding_File.csv", parse_dates = ['ADMIT_DT','FORM_DTTM',
                                                                                                'FORM_UPDATE_DTTM','DOB'])


#subsetting to include Peer Recovery Program episodes
df = df[df['EPISODE'] =='IFPR - SBIRT'].sort_values(by = ['FORM_DTTM','MRN']) # <- sorting df

In [3]:
#Renaming column to create ClientID
df.rename(columns = {'CSN':'ClientID'}, inplace = True)

In [4]:
# function to return all values greater than 30 as 30 to be utilized for days columns
def thirty(x):
    if x > 30:
        return 30 #-- returns value 30 if number greater than 30
    else:
        return x #-- will return same value if number not greater than 30

In [5]:
#Creating function for interview type
def interview_type_code(row):
    if row['INTERVIEW_TYPE'] == 'Intake':
        return 1
    elif row['INTERVIEW_TYPE'] == '6-month follow-up':
        return 2
    elif row['INTERVIEW_TYPE'] == '3-month follow-up':
        return 4
    elif row['INTERVIEW_TYPE'] == 'Discharge':
        return 5
    
#Creating new column via applying interview_type_code function to values in interview type column
df['InterviewType'] = df.apply(interview_type_code, axis = 1)

## Creating GPRA 6

In [6]:
# Creating function to create column FLWPClientContacted
def FLWP_ClientContacted (row):
    if row['CLIENT_ABLE_TO_BE_CONTACTED_FOR_FOLLOW_UP'] == 'Yes':
        return 1
    elif row['CLIENT_ABLE_TO_BE_CONTACTED_FOR_FOLLOW_UP'] == 'No':
        return 0
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] in [2,5]:
        return FLWP_ClientContacted (row)
    else:
        return pd.NA
    
    
#Creating column FLWPClientContacted via applying the function FLWP_ClientContacted (x)
df['FLWPClientContacted'] = df.apply(interview_type, axis = 1)

In [7]:
#Creating function to create column ReceivingServices
def Receiving_Services (row):
    if row['CLIENT_STILL_RECEIVING_PROGRAM_SERVICES'] == 'Yes':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] in [2,5]:
        return Receiving_Services (row)
    else:
        return pd.NA
    
#Creating column ReceivingServices via applying the function Receiving_Services (x)
df['ReceivingServices'] = df.apply(interview_type, axis = 1)

In [8]:
#Creating dictionary to rename columns
dictionary10 = {'FOLLOW_UP_STATUS_OF_CLIENT':'FLWPStatus','OTHER_FOLLOW_UP_STATUS':'FLWPStatusSpec',
                'CLIENT_DISCHARGE_DATE':'DischargeDate','CLIENT_DISCHARGE_STATUS':'DischargeStatusCompl',
                'REASON_FOR_TERMINATION':'DischargeStatusTermReason',
                'OTHER_REASON_FOR_TERMINATION':'OtherDischargeStatTermRsnSpec',
                'HIV_TEST_ORDERED_BY_PROGRAM':'jHIVTest','CLIENT_REFERRERD_FOR_HIV_TEST':'jHIVTestResult'}

#Renaming columns
df.rename(columns = dictionary10,inplace = True)

#Creating variables to replace null values
null_col10 = ['jHIVTest','jHIVTestResult']

#Replacing null values with -9
df[null_col10] = df[null_col10].fillna(-9)

# Data Cleaning
df['FLWPStatus'] = df['FLWPStatus'].str[:2]
# df['FLWPStatusSpec'] = df['FLWPStatusSpec'].str.title()
df['DischargeStatusCompl'] = df['DischargeStatusCompl'].str[1:2]
# df['DischargeStatusTermReason'] = df['DischargeStatusTermReason'].str[1]
# df['OtherDischargeStatTermRsnSpec'] = df['OtherDischargeStatTermRsnSpec'].str.title()
df['jHIVTest'] = df['jHIVTest'].replace('No',0)
df['jHIVTestResult'] = df['jHIVTestResult'].replace('Yes',1)
df['jHIVTestResult'] = df['jHIVTestResult'].replace('No',0)

#Converting df['DischargeDate'] to datetime data type
df['DischargeDate'] = pd.to_datetime(df['DischargeDate'])

#C,overting to the mm/dd/yyyy format
df['DischargeDate'] = df['DischargeDate'].dt.strftime('%m/%d/%Y')

In [87]:
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['jHIVTest']
    else:
        return pd.NA
    
df['jHIVTest'] = df.apply(interview_type, axis =1 )


def hiv_param(row):
    if row['jHIVTest'] == 0:
        return row['jHIVTestResult']
    elif row['jHIVTest'] == 1:
        return -1
    elif row['jHIVTest'] == -9:
        return -1
    else:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return hiv_param(row)
    else:
        return pd.NA
    
df['jHIVTestResult'] = df.apply(interview_type, axis =1 )

In [10]:
def dischargeStatusTermReason(row):
    if row['DischargeStatusCompl'] == '1':
        return -1
    elif row['DischargeStatusCompl'] == '2':
        return row['DischargeStatusTermReason']
    
def interview_type(row):
    if row['InterviewType'] in [2,5]:
        return dischargeStatusTermReason(row)
    else:
        return pd.NA

df['DischargeStatusTermReason'] = df.apply(interview_type, axis = 1)

In [11]:
# filtering FLWPStatus column based on interview type
def interview_type(row):
    if row['InterviewType'] in [2,5]:
        return row['FLWPStatus']
    else:
        return pd.NA
    
df['FLWPStatus'] = df.apply(interview_type, axis = 1)

In [12]:
def FLWPStatus (row):
    if row['FLWPClientContacted'] == 1:
        return row['FLWPStatus']
    elif row['FLWPClientContacted'] == 0:
        return 32
    
def interview_type(row):
    if row['InterviewType'] in [2,5]:
        return FLWPStatus (row)
    else:
        pd.NA
    
df['FLWPStatus'] = df.apply(interview_type, axis = 1)

In [13]:
def FLWPStatusSpec (row):
    if row['FLWPStatus'] == 32:
        return 'None'
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] in [2,5]:
        return  FLWPStatusSpec (row)
    else:
        pd.NA
    
df['FLWPStatusSpec'] = df.apply(interview_type, axis = 1)

In [14]:
#Replacing null values with -9
df['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'] = df['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'].fillna(-9)

#Creating function to create column ODPreventionToolsProvided
def ODPrevention_ToolsProvided (row):
    if row['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'] == 'Neither':
        return 4
    elif row['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'] == 'Naloxone':
        return 1
    elif row['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'] == -9:
        return -9
    
def interview_type(row):
    if row['InterviewType'] in [2,5]:
        return  ODPrevention_ToolsProvided (row)
    else:
        pd.NA
    
#Creating column ODPreventionToolsProvided via applying the function ODPrevention_ToolsProvided (x)
df['ODPreventionToolsProvided'] = df.apply(interview_type, axis = 1)

In [15]:
#Replacing null values with -9
df['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] = df['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'].fillna(-9)

#Creating function to create column VaccinatedStatusCOVID19
def Vaccinated_StatusCOVID19 (row):
    if row['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] == 'Yes':
        return 1
    elif row['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] == 'Refused to answer':
        return -7
    elif row['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] == 'No, client refused vaccination':
        return 4
    elif row['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] == 'No, partially vaccinated with no plan to receive the subsequent vaccination':
        return 3
    elif row['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] == -9:
        return -9
    
def interview_type(row):
    if row['InterviewType'] in [2,5]:
        return  Vaccinated_StatusCOVID19 (row)
    else:
        pd.NA
    
#Creating column VaccinatedStatusCOVID19 via applying the function Vaccinated_StatusCOVID19
df['VaccinatedStatusCOVID19'] = df.apply(interview_type, axis = 1)

In [16]:
#Creating column SvcInpatient
df['SvcInpatient'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcInpatient']
    else:
        pd.NA
    
df['SvcInpatient'] = df.apply(interview_type, axis = 1)

#Creating column SvcOutpatient
df['SvcOutpatient'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOutpatient']
    else:
        pd.NA
    
df['SvcOutpatient'] = df.apply(interview_type, axis = 1)

In [17]:
#Creating columns in which no services were provided
df['SvcMethadone'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcMethadone']
    else:
        pd.NA
    
df['SvcMethadone'] = df.apply(interview_type, axis = 1)

df['SvcBuprenorphine'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcBuprenorphine']
    else:
        pd.NA
    
df['SvcBuprenorphine'] = df.apply(interview_type, axis = 1)

df['SvcNaltrexoneShortActing'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcNaltrexoneShortActing']
    else:
        pd.NA
    
df['SvcNaltrexoneShortActing'] = df.apply(interview_type, axis = 1)

df['SvcNaltrexoneLongActing'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcNaltrexoneLongActing']
    else:
        pd.NA
    
df['SvcNaltrexoneLongActing'] = df.apply(interview_type, axis = 1)

df['SvcDisulfiram'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcDisulfiram']
    else:
        pd.NA
    
df['SvcDisulfiram'] = df.apply(interview_type, axis = 1)

df['SvcAcamprosate'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcAcamprosate']
    else:
        pd.NA
    

df['SvcAcamprosate'] = df.apply(interview_type, axis = 1)

df['SvcNicotineReplacement'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcNicotineReplacement']
    else:
        pd.NA
    

df['SvcNicotineReplacement'] = df.apply(interview_type, axis = 1)

df['SvcBupropion'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcBupropion']
    else:
        pd.NA
    

df['SvcBupropion'] = df.apply(interview_type, axis = 1)

df['SvcVarenicline'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcVarenicline']
    else:
        pd.NA
    

df['SvcVarenicline'] = df.apply(interview_type, axis = 1)

df['SvcResidentialRehab'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcResidentialRehab']
    else:
        pd.NA
    

df['SvcResidentialRehab'] = df.apply(interview_type, axis = 1)

df['SvcHospitalInpatient'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcHospitalInpatient']
    else:
        pd.NA
    

df['SvcHospitalInpatient'] = df.apply(interview_type, axis = 1)


df['SvcFreeStandingRes'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcFreeStandingRes']
    else:
        pd.NA
    

df['SvcFreeStandingRes'] = df.apply(interview_type, axis = 1)

df['SvcAmbulatoryDetox'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcAmbulatoryDetox']
    else:
        pd.NA
    

df['SvcAmbulatoryDetox'] = df.apply(interview_type, axis = 1)

df['SvcAfterCare'] = 0

def interview_type(row):
    if row['InterviewType'] ==  5:
        return  row['SvcAfterCare']
    else:
        pd.NA
    

df['SvcAfterCare'] = df.apply(interview_type, axis = 1)

In [18]:
#Creating columns with blank values
df['SvcOtherModalities'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOtherModalities']
    else:
        pd.NA

df['SvcOtherModalities'] = df.apply(interview_type, axis = 1)

df['SvcOtherModalitesSpec'] = pd.NA

In [19]:
#Creating column SvcBriefTreatment
df['SvcBriefTreatment'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcBriefTreatment']
    else:
        pd.NA
    

df['SvcBriefTreatment'] = df.apply(interview_type, axis = 1)

In [20]:
#Creating dictionary to rename columns 
dictionary8 = {'REFERRAL_TO_TREATMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcReferralTreatment', 'ASSESSMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcAssessment', 
               'TREATMENT_PLANNING_NUMBER_OF_SESSIONS_RECEIVED':'SvcTreatmentPlanning', 
               'INDIVIDUAL_COUNSELING_NUMBER_OF_SESSIONS_RECEIVED':'SvcIndividualCouns'}

#Renaming 
df.rename(columns = dictionary8,inplace = True)

#Creating variable to replace null values
col_null14 = ['SvcReferralTreatment', 'SvcTreatmentPlanning', 'SvcIndividualCouns', 'SvcAssessment']

#Replacing null values with 0
df[col_null14] = df[col_null14].fillna(0)

In [21]:
# filtering columns based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcReferralTreatment']
    else:
        pd.NA
    

df['SvcReferralTreatment'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcTreatmentPlanning']
    else:
        pd.NA
    

df['SvcTreatmentPlanning'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcIndividualCouns']
    else:
        pd.NA
    

df['SvcIndividualCouns'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcAssessment']
    else:
        pd.NA
    
df['SvcAssessment'] = df.apply(interview_type, axis = 1)

In [22]:
#Creating columns with missing values 
df['SvcGroupCouns'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcGroupCouns']
    else:
        pd.NA
    
df['SvcGroupCouns'] = df.apply(interview_type, axis = 1)


df['SvcContingencyManagement'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcContingencyManagement']
    else:
        pd.NA
    
df['SvcContingencyManagement'] = df.apply(interview_type, axis = 1)


df['SvcCommunityReinforcement'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcCommunityReinforcement']
    else:
        pd.NA
    
df['SvcCommunityReinforcement'] = df.apply(interview_type, axis = 1)


df['SvcCBT'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcCBT']
    else:
        pd.NA
    
df['SvcCBT'] = df.apply(interview_type, axis = 1)

df['SvcFamilyMarriageCouns'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcFamilyMarriageCouns']
    else:
        pd.NA
    
df['SvcFamilyMarriageCouns'] = df.apply(interview_type, axis = 1)

df['SvcCoOccurring'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcCoOccurring']
    else:
        pd.NA
    
df['SvcCoOccurring'] = df.apply(interview_type, axis = 1)

df['SvcPharmacological'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcPharmacological']
    else:
        pd.NA
    
df['SvcPharmacological'] = df.apply(interview_type, axis = 1)

df['SvcHIVAIDSCouns'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcHIVAIDSCouns']
    else:
        pd.NA
    
df['SvcHIVAIDSCouns'] = df.apply(interview_type, axis = 1)

df['SvcCulturalInterventions'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcCulturalInterventions']
    else:
        pd.NA
    
df['SvcCulturalInterventions'] = df.apply(interview_type, axis = 1)

df['SvcOtherClinicalCouns'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOtherClinicalCouns']
    else:
        pd.NA
    
df['SvcOtherClinicalCouns'] = df.apply(interview_type, axis = 1)

df['SvcOtherClinicalCounsSpec'] = " "

df['SvcChildCare'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcChildCare']
    else:
        pd.NA
    
df['SvcChildCare'] = df.apply(interview_type, axis = 1)


In [23]:
#Creating dictionary to rename
dictionary11 = {'CASE_MANAGEMENT_NUMBER_OF_DAYS_RECEIVED':'SvcCaseManagement', 
                'INTENSIVE_OUTPATIENT_TREATMENT_NUMBER_OF_DAYS_RECEIVED':'SvcIntensiveOutpatient', 
                'OUTREACH_NUMBER_OF_DAYS_RECEIVED':'SvcOutreach',
                'RECOVERY_SUPPORT_NUMBER_OF_DAYS_RECEIVED':'SvcRecoverySupport',
               'SCREENING_NUMBER_OF_SESSIONS_RECEIVED':'SvcScreening',
                'BRIEF_INTERVENTION_NUMBER_OF_SESSIONS_RECEIVED':'SvcBriefIntervention',
               'FAMILY_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcFamilyServices', 
                'PRE_EMPLOYMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcPreEmployment'}

#Renaming columns 
df.rename(columns = dictionary11,inplace = True)

#Replacing null values with 0
col_fill0 =['SvcCaseManagement', 'SvcIntensiveOutpatient', 'SvcOutreach','SvcRecoverySupport',
            'SvcScreening','SvcBriefIntervention','SvcFamilyServices', 'SvcPreEmployment']

df[col_fill0] = df[col_fill0].fillna(0)

In [24]:
def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcCaseManagement']
    else:
        pd.NA
    
df['SvcCaseManagement'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcIntensiveOutpatient']
    else:
        pd.NA
    
df['SvcIntensiveOutpatient'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOutreach']
    else:
        pd.NA
    
df['SvcOutreach'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcRecoverySupport']
    else:
        pd.NA
    
df['SvcRecoverySupport'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcRecoverySupport']
    else:
        pd.NA
    
df['SvcRecoverySupport'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcScreening']
    else:
        pd.NA
    
df['SvcScreening'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcBriefIntervention']
    else:
        pd.NA
    
df['SvcBriefIntervention'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcFamilyServices']
    else:
        pd.NA
    
df['SvcFamilyServices'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcPreEmployment']
    else:
        pd.NA
    
df['SvcPreEmployment'] = df.apply(interview_type, axis = 1)

In [25]:
#Renaming columns 
df.rename(columns = {'EMPLOYMENT_COACHING_NUMBER_OF_SESSIONS_RECEIVED':'SvcEmploymentCoaching', 
                     'INDIVIDUAL_SERVICES_COORDINATION_NUMBER_OF_SESSIONS_RECEIVED':'SvcIndividualCoord', 
                     'TRANSPORTATION_NUMBER_OF_SESSIONS_RECEIVED': 'SvcTransportation'}, inplace = True)

#Creating variable for columns
col1 = ['SvcEmploymentCoaching','SvcIndividualCoord','SvcTransportation']

#Replacing null values
df[col1] = df[col1].fillna(0)

In [26]:
def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcEmploymentCoaching']
    else:
        pd.NA
    
df['SvcEmploymentCoaching'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcIndividualCoord']
    else:
        pd.NA
    
df['SvcIndividualCoord'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcTransportation']
    else:
        pd.NA
    
df['SvcTransportation'] = df.apply(interview_type, axis = 1)


In [27]:
#Creating column with missing values
df['SvcHIVAIDSServices'] = 0

df.rename(columns = {'TRANSITIONAL_DRUG_FREE_HOUSING_NUMBER_OF_SESSIONS_RECEIVED':'SvcDrugFreeHousing','HOUSING_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcHousingSupport',
                     'HEALTH_INSURANCE_ENROLLMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcHealthInsuranceEnrollment',
                     'OTHER_CASE_MANAGEMENT_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherCaseMgmt',
                     'OTHER_CASE_MANAGEMENT_SERVICES_RECEIVED': 'SvcOtherCaseMgmtSpec'}, inplace = True)

#Creating variable 'col' to replace missing values
col2 = ['SvcDrugFreeHousing','SvcHousingSupport','SvcHealthInsuranceEnrollment','SvcOtherCaseMgmt']

#Replacing missing values with 0
df[col2] = df[col2].fillna(0)

#Editing string values
# df['SvcOtherCaseMgmtSpec'] = df['SvcOtherCaseMgmtSpec'].str.title()


In [28]:
def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcHIVAIDSServices']
    else:
        pd.NA
    
df['SvcHIVAIDSServices'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcDrugFreeHousing']
    else:
        pd.NA
    
df['SvcDrugFreeHousing'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcHousingSupport']
    else:
        pd.NA
    
df['SvcHousingSupport'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcHealthInsuranceEnrollment']
    else:
        pd.NA
    
df['SvcHealthInsuranceEnrollment'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOtherCaseMgmt']
    else:
        pd.NA
    
df['SvcOtherCaseMgmt'] = df.apply(interview_type, axis = 1)


In [29]:
#Renaming columns 
df.rename(columns = {'MEDICAL_CARE_NUMBER_OF_SESSIONS_RECEIVED':'SvcMedicalCare'}, inplace = True)

#Replacing missing values with 0
df['SvcMedicalCare'] = df['SvcMedicalCare'].fillna(0) 

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcMedicalCare']
    else:
        pd.NA
    
df['SvcMedicalCare'] = df.apply(interview_type, axis = 1)


#Creating columns with missing values 
df['SvcAlcoholDrugTesting'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcAlcoholDrugTesting']
    else:
        pd.NA
    
df['SvcAlcoholDrugTesting'] = df.apply(interview_type, axis = 1)

df['SvcOBGYN'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOBGYN']
    else:
        pd.NA
    
df['SvcOBGYN'] = df.apply(interview_type, axis = 1)

df['SvcHIVAIDSMedical'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcHIVAIDSMedical']
    else:
        pd.NA
    
df['SvcHIVAIDSMedical'] = df.apply(interview_type, axis = 1)

df['SvcHepatitisSupport'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcHepatitisSupport']
    else:
        pd.NA
    
df['SvcHepatitisSupport'] = df.apply(interview_type, axis = 1)

df['SvcOtherSTISupport'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOtherSTISupport']
    else:
        pd.NA
    
df['SvcOtherSTISupport'] = df.apply(interview_type, axis = 1)


In [30]:
#Renaming columns
df.rename(columns = {'DENTAL_CARE_NUMBER_OF_SESSIONS_RECEIVED':'SvcDental', 
                     'OTHER_MEDICAL_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherMedical',
                     'OTHER_MEDICAL_SERVICES_RECEIVED':'SvcOtherMedicalSpec',
                     'CONTINUING_CARE_NUMBER_OF_SESSIONS_RECEIVED':'SvcContinuingCare',
                     'RELAPSE_PREVENTION_NUMBER_OF_SESSIONS_RECEIVED':'SvcRelapsePrevention'}, inplace = True)

#Creating variable col1 to replace missing values with 0
col3 = ['SvcDental','SvcOtherMedical','SvcContinuingCare','SvcRelapsePrevention']


#Replacing missing values with 0
df[col3] = df[col3].fillna(0) 

#Data Cleaning
# df['SvcOtherMedicalSpec'] = df['SvcOtherMedicalSpec'].str.title() 

In [31]:
def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcDental']
    else:
        pd.NA
    
df['SvcDental'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOtherMedical']
    else:
        pd.NA
    
df['SvcOtherMedical'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcContinuingCare']
    else:
        pd.NA
    
df['SvcContinuingCare'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcRelapsePrevention']
    else:
        pd.NA
    
df['SvcRelapsePrevention'] = df.apply(interview_type, axis = 1)


In [32]:
#Renaming columns
df.rename(columns = {'RECOVERY_COACHING_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryCoaching','SELF_HELP_MUTUAL_SUPPORT_GROUP_NUMBER_OF_SESSIONS_RECEIVED':'SvcSelfHelpSupport','SPIRITUAL_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcSpiritualSupport','OTHER_AFTER_CARE_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherAfterCare','OTHER_AFTER_CARE_SERVICES_RECEIVED':'SvcOtherAfterCareSpec'}, inplace = True)

#Creating variable col2 to replace missing values
col4 = ['SvcRecoveryCoaching','SvcSelfHelpSupport','SvcSpiritualSupport','SvcOtherAfterCare']

#Replacing missing values with 0
df[col4] = df[col4].fillna(0)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcRecoveryCoaching']
    else:
        pd.NA
    
df['SvcRecoveryCoaching'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcSelfHelpSupport']
    else:
        pd.NA
    
df['SvcSelfHelpSupport'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcSpiritualSupport']
    else:
        pd.NA
    
df['SvcSpiritualSupport'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOtherAfterCare']
    else:
        pd.NA
    
df['SvcOtherAfterCare'] = df.apply(interview_type, axis = 1)

In [33]:
#Renaming column
df.rename(columns = {'SUBSTANCE_MISUSE_EDUCATION_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcSubstanceAbuseEdu'}, inplace = True )

#Replacing missing values with 0
df['SvcSubstanceAbuseEdu'] = df['SvcSubstanceAbuseEdu'].fillna(0) 

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcSubstanceAbuseEdu']
    else:
        pd.NA
    
df['SvcSubstanceAbuseEdu'] = df.apply(interview_type, axis = 1)

df['SvcHIVAIDSEdu'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcHIVAIDSEdu']
    else:
        pd.NA
    
df['SvcHIVAIDSEdu'] = df.apply(interview_type, axis = 1)


df['SvcHepatitisEdu'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcHepatitisEdu']
    else:
        pd.NA
    
df['SvcHepatitisEdu'] = df.apply(interview_type, axis = 1)

df['SvcOtherSTIEdu'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOtherSTIEdu']
    else:
        pd.NA
    
df['SvcOtherSTIEdu'] = df.apply(interview_type, axis = 1)

df['SvcNaloxoneTraining'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcNaloxoneTraining']
    else:
        pd.NA
    
df['SvcNaloxoneTraining'] = df.apply(interview_type, axis = 1)

df['SvcFentanylTestStripTraining'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcFentanylTestStripTraining']
    else:
        pd.NA
    
df['SvcFentanylTestStripTraining'] = df.apply(interview_type, axis = 1)

df['SvcOtherEdu'] = 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOtherEdu']
    else:
        pd.NA
    
df['SvcOtherEdu'] = df.apply(interview_type, axis = 1)


df['SvcOtherEduSpec'] = " "

In [34]:
#Creating copy of RECOVERY_HOUSING_NUMBER_OF_SESSIONS_RECEIVED column
df['RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED_1'] = df['RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED']

#Renaming columns 
df.rename(columns = {'PEER_COACHING_OR_MENTORING_NUMBER_OF_SESSIONS_RECEIVED':'SvcPeerCoaching', 
                     'VOCATIONAL_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcVocational',
                     'RECOVERY_HOUSING_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryHousing',
                     'RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryPlanningRSS',
                     'RECOVERY_SUPPORT_SPECIFIC_CASE_MANAGEMENT_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryCaseManagement',
                     'ALCOHOL_AND_DRUG_FREE_SOCIAL_ACTIVITIES_NUMBER_OF_SESSIONS_RECEIVED':'SvcDrugFreeSocial'}, inplace = True)

#Creating variable col3
col5 = ['SvcPeerCoaching','SvcVocational','SvcRecoveryHousing','SvcRecoveryCaseManagement','SvcDrugFreeSocial'] 

#Replacing missing values with 0
df[col5] = df[col5].fillna(0)

#inputting values in the 'SvcRecoveryPlanningRSS' column
df['SvcRecoveryPlanningRSS'] = df['SvcRecoveryPlanningRSS'].fillna(0)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcRecoveryPlanningRSS']
    else:
        pd.NA
    
df['SvcRecoveryPlanningRSS'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcPeerCoaching']
    else:
        pd.NA
    
df['SvcPeerCoaching'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcVocational']
    else:
        pd.NA
    
df['SvcVocational'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcRecoveryHousing']
    else:
        pd.NA
    
df['SvcRecoveryHousing'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcRecoveryCaseManagement']
    else:
        pd.NA
    
df['SvcRecoveryCaseManagement'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcDrugFreeSocial']
    else:
        pd.NA
    
df['SvcDrugFreeSocial'] = df.apply(interview_type, axis = 1)


In [35]:
#Renaming columns
df.rename(columns = {'INFORMATION_AND_REFERRAL_NUMBER_OF_SESSIONS_RECEIVED':'SvcInformationReferral','OTHER_RECOVERY_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherRecoverySupport','OTHER_RECOVERY_SUPPORT_SERVICES_RECEIVED':'SvcOtherRecoverySupportSpec','OTHER_PEER_TO_PEER_RECOVERY_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherPeerRecovery','OTHER_PEER_TO_PEER_RECOVERY_SUPPORT_RECEIVED':'SvcOtherPeerRecoverySpec'}, inplace = True)

#Creating variable col4 to replace missing values
col6 = ['SvcInformationReferral','SvcOtherRecoverySupport','SvcOtherPeerRecovery']

#Replacing missing values with 0
df[col6] = df[col6].fillna(0)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcInformationReferral']
    else:
        pd.NA
    
df['SvcInformationReferral'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOtherRecoverySupport']
    else:
        pd.NA
    
df['SvcOtherRecoverySupport'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return  row['SvcOtherPeerRecovery']
    else:
        pd.NA
    
df['SvcOtherPeerRecovery'] = df.apply(interview_type, axis = 1)

In [36]:
#Replacing null values with -9
df['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'] = df['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'].fillna(-9)

#Creating function to create column AttendedMostPlannedServices
def AttendedMost_PlannedServices (row):
    if row['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'] == 'Yes':
        return 1
    elif row['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'] == 'No':
        return 0
    elif row['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'] == -9:
        return -9

    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  AttendedMost_PlannedServices (row)
    else:
        pd.NA

df['AttendedMostPlannedServices'] = df.apply(interview_type,axis = 1)

In [37]:
#Replacing null values with -9
df['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'] = df['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'].fillna(-9)

#Creating function to create column ReceivedTelehealthServices
def Received_Telehealth_Services (row):
    if row['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'] == 'Yes':
        return 1
    elif row['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'] == 'No':
        return 0
    elif row['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'] == -9:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  Received_Telehealth_Services (row)
    else:
        pd.NA

df['ReceivedTelehealthServices'] = df.apply(interview_type, axis = 1)

In [38]:
#Replacing null values with -9
df['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] = df['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'].fillna(-9)

#Creating a function to create column PreviouslyDiagnosedOUD
def Previously_DiagnosedOUD (row):
    if row['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] == 'Yes':
        return 1
    elif row['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] == 'No [SKIP TO QUESTION 5]':
        return 0
    elif row['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] == -9:
        return -9

    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  Received_Telehealth_Services (row)
    else:
        pd.NA

df['PreviouslyDiagnosedOUD'] = df.apply(interview_type, axis = 1)

In [39]:
#Replacing null values with -9
# df['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] = df['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]
    
#Creating function to create column kOUDMedMethadone
def kOUD_Med_Methadone (row):
    if row ['PreviouslyDiagnosedOUD']== -9 or row ['PreviouslyDiagnosedOUD']== 0 :
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Methadone':
        return 1
    else:
        return 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  kOUD_Med_Methadone (row)
    else:
        pd.NA

    
#Creating column kOUDMedMethadone via applying the function kOUD_Med_Methadone (x)
df['kOUDMedMethadone'] = df.apply(interview_type, axis = 1)

In [40]:
# creating column kOUDMedMethadoneDays
df.rename(columns = {'METHADONE_NUMBER_OF_DAYS':'kOUDMedMethadoneDays'}, inplace = True)

def kOUDMedMethadoneDays (row):
    if row['kOUDMedMethadone'] == 1 :
        return row['kOUDMedMethadoneDays']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kOUDMedMethadoneDays (row)
    else:
        pd.NA
    
df['kOUDMedMethadoneDays'] = df.apply(interview_type, axis = 1)

In [41]:
#Creating function to create column kOUDMedBuprenorphine
def kOUDMed_Buprenorphine (row):
    if row['PreviouslyDiagnosedOUD'] == -9 or  row ['PreviouslyDiagnosedOUD']== 0 :
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Buprenorphine':
        return 1
    else:
        return 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  kOUDMed_Buprenorphine (row)
    else:
        pd.NA
        
#Creating column kOUDMedMethadone via applying the function kOUDMed_Buprenorphine (x)
df['kOUDMedBuprenorphine'] = df.apply(interview_type, axis = 1)

In [42]:
# creating column kOUDMedBuprenorphineDays
df.rename(columns = {'BUPRENORPHINE_NUMBER_OF_DAYS':'kOUDMedBuprenorphineDays'}, inplace = True)

def kOUDMedBuprenorphineDays (row):
    if row['kOUDMedBuprenorphine'] == 1:
        return row['kOUDMedBuprenorphineDays']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kOUDMedBuprenorphineDays (row)
    else:
        pd.NA    

df['kOUDMedBuprenorphineDays'] = df.apply(interview_type, axis = 1)


In [43]:
#Creating function to create column kOUDMedNaltrexone
def kOUDMed_Naltrexone (row):
    if row['PreviouslyDiagnosedOUD']== -9 or row ['PreviouslyDiagnosedOUD']== 0 :
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Naltrexone':
        return 1
    else:
        return 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  kOUDMed_Naltrexone (row)
    else:
        pd.NA 
    
#Creating column kOUDMedNaltrexone via applying the function kOUDMed_Naltrexone (x)
df['kOUDMedNaltrexone'] = df.apply(interview_type, axis = 1)

In [44]:
# creating column kOUDMedNaltrexoneDays
def kOUDMedNaltrexoneDays (row):
    if row['kOUDMedNaltrexone'] == 1:
        return row['NALTREXONE_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kOUDMedNaltrexoneDays (row)
    else:
        pd.NA 

df['kOUDMedNaltrexoneDays'] = df.apply(interview_type, axis = 1)

In [45]:
# creating column kOUDMedXRNaltrexone
def kOUDMedXRNaltrexone(row):
    if row['PreviouslyDiagnosedOUD']== -9 or row ['PreviouslyDiagnosedOUD']== 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Extended-release Naltrexone':
            return 1
    else:
            return 0
        
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kOUDMedXRNaltrexone(row)
    else:
        pd.NA 

df['kOUDMedXRNaltrexone'] = df.apply(interview_type, axis = 1)

In [46]:
# creating column kOUDMedXRNaltrexoneDoses
def kOUDMedXRNaltrexoneDoses(row):
    if row['kOUDMedXRNaltrexone'] == 1:
        return row['NUMBER_OF_DAYS_RECEIVED_EXTENDED_RELEASE_NALTREXONE']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kOUDMedXRNaltrexoneDoses(row)
    else:
        pd.NA

df['kOUDMedXRNaltrexoneDoses'] = df.apply(interview_type, axis = 1)


In [47]:
# creating the column kOUDMedNotReceived
def kOUDMedNotReceived(row):
    if row['PreviouslyDiagnosedOUD']== -9 or row ['PreviouslyDiagnosedOUD']== 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive an FDA-approved medication for a diagnosed opioid use disorder [SKIP TO QUESTION 5]':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kOUDMedNotReceived(row)
    else:
        pd.NA    

df['kOUDMedNotReceived'] = df.apply(interview_type, axis = 1)

In [48]:
# creating column OUDMedTakenAsPrescribed
def OUDMedTakenAsPrescribed (row):
    if row['PreviouslyDiagnosedOUD']== -9 or row ['PreviouslyDiagnosedOUD']== 0:
        return -1
    elif row['kOUDMedMethadone'] == 1:
        return 1
    elif row['kOUDMedBuprenorphine'] == 1:
        return 1
    elif row['kOUDMedNaltrexone'] == 1:
        return 1
    elif row['kOUDMedXRNaltrexone'] == 1:
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  OUDMedTakenAsPrescribed (row)
    else:
        pd.NA 
    
df['OUDMedTakenAsPrescribed'] = df.apply(interview_type, axis = 1)

In [49]:
df['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'] = df['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'].fillna(-9)
# creating column PreviouslyDiagnosedAUD
def PreviouslyDiagnosedAUD (row):
    if row['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'] == 'Yes':
        return 1
    elif row['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'] == 'No':
        return 0
    else:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  PreviouslyDiagnosedAUD (row)
    else:
        pd.NA 
    
df['PreviouslyDiagnosedAUD'] = df.apply(interview_type, axis = 1)

In [50]:
# df['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] = df['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]

def kAUDMedNaltrexone (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0 :
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Naltrexone':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kAUDMedNaltrexone (row)
    else:
        pd.NA 
    
df['kAUDMedNaltrexone'] = df.apply(interview_type, axis = 1)

In [51]:
def kAUDMedNaltrexoneDays (row):
    if row['kAUDMedNaltrexone'] == 1:
        return row['NALTREXONE_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kAUDMedNaltrexoneDays (row)
    else:
        pd.NA
    
df['kAUDMedNaltrexoneDays'] = df.apply(interview_type, axis = 1)

In [52]:
def kAUDMedXRNaltrexone (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Extended-release naltrexone':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kAUDMedXRNaltrexone (row)
    else:
        pd.NA
    
df['kAUDMedXRNaltrexone'] = df.apply(interview_type, axis = 1)

In [53]:
def kAUDMedXRNaltrexoneDoses (row):
    if row['kAUDMedXRNaltrexone'] == 1:
        return row['EXTENDED_RELEASE_NALTREXONE_NUMBER_OF_DAYS']
    else:
        pass

def interview_type(row):
    if row['InterviewType'] == 5:
        return  kAUDMedXRNaltrexoneDoses (row)
    else:
        pd.NA    
    
df['kAUDMedXRNaltrexoneDoses'] = df.apply(interview_type, axis = 1)

In [54]:
def kAUDMedDisulfiram (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Disulfiram':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   kAUDMedDisulfiram (row)
    else:
        pd.NA      

    
df['kAUDMedDisulfiram'] = df.apply(interview_type , axis = 1)

In [55]:
def kAUDMedDisulfiramDays (row):
    if row['kAUDMedDisulfiram'] == 1:
        return row['NUMBER_OF_DAYS_RECEIVED_DISULFIRAM']
    else:
        pass
    

def interview_type(row):
    if row['InterviewType'] == 5:
        return   kAUDMedDisulfiramDays (row)
    else:
        pd.NA      
    

df['kAUDMedDisulfiramDays'] = df.apply(interview_type, axis = 1)

In [56]:
def kAUDMedAcamprosate (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Acamprosate':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   kAUDMedAcamprosate (row)
    else:
        pd.NA
        
        
df['kAUDMedAcamprosate'] = df.apply(interview_type, axis = 1)

In [57]:
def kAUDMedAcamprosateDays (row):
    if row['kAUDMedAcamprosate'] == 1:
        return row['NUMBER_OF_DAYS_RECEIVED_ACAMPROSATE']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   kAUDMedAcamprosateDays (row)
    else:
        pd.NA    

df['kAUDMedAcamprosateDays'] = df.apply(interview_type, axis = 1)

In [58]:
#Creating function to create column kAUDMedNotReceived
def  kAUDMed_NotReceived (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive an FDA-approved medication for an alcohol use disorder [SKIP TO QUESTION 6]':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   kAUDMed_NotReceived (row)
    else:
        pd.NA 
    
#Creating column kAUDMedNotReceived via applying the function kAUDMed_NotReceived (x)
df['kAUDMedNotReceived'] = df.apply(interview_type, axis = 1)

In [59]:
#Creating column with missing values
def AUDMedTakenAsPrescribed (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0:
        return -1
    elif row['kAUDMedNaltrexone'] == 1:
        return 1
    elif row['kAUDMedXRNaltrexone'] == 1:
        return 1
    elif row['kAUDMedDisulfiram'] == 1:
        return 1
    elif row['kAUDMedAcamprosate'] == 1:
        return 1
    else:
        pd.NA
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  AUDMedTakenAsPrescribed (row)
    else:
        pd.NA 
    
df['AUDMedTakenAsPrescribed'] = df.apply(AUDMedTakenAsPrescribed, axis = 1)

In [60]:
df[['InterviewType','AUDMedTakenAsPrescribed']]

,InterviewType,AUDMedTakenAsPrescribed
12,1.0,NaN
10,1.0,NaN
9,2.0,NaN
8,5.0,-1.0
5,NaN,NaN
11,NaN,NaN
0,5.0,-1.0
7,1.0,NaN
4,5.0,-1.0
14,1.0,NaN


In [61]:
#Replacing null values with -9
df['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] = df['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'].fillna(-9)

#Creating function to create column PreviouslyDiagnosedStUD
def Previously_DiagnosedStUD (row):
    if row['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] == 'Yes':
        return 1
    elif row['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] == 'No [SKIP TO QUESTION 7]':
        return 0
    elif row['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] == -9:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   Previously_DiagnosedStUD (row)
    else:
        pd.NA     

    
#Creating column PreviouslyDiagnosedStUD via applying the function Previously_DiagnosedStUD (x)
df['PreviouslyDiagnosedStUD'] = df.apply(interview_type, axis = 1)

In [62]:
#Replacing null valules with -9
# df['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] = df['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]

#Creating function to create column kStUDIntContMgmt
def kStUDIntContMgmt (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0 :
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Contingency Management':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return    kStUDIntContMgmt (row)
    else:
        pd.NA   

#Creating column kStUDIntCommReinf via applying the function kStUDInt_CommReinf (x)
df['kStUDIntContMgmt'] = df.apply(interview_type, axis = 1)

In [63]:
def kStUDIntContMgmtDays (row):
    if row['kStUDIntContMgmt'] == 1:
        return row['CONTINGENCY_MANAGEMENT_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return     kStUDIntContMgmtDays (row)
    else:
        pd.NA  
        
    
df['kStUDIntContMgmtDays'] = df.apply(interview_type, axis = 1)

In [64]:
#Creating function to create column kStUDIntCommReinf
def kStUDIntCommReinf (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0:
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Community Reinforcement':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return     kStUDIntCommReinf (row)
    else:
        pd.NA  
        

#Creating column kStUDIntCommReinf via applying the function kkStUDIntCommReinf (x)
df['kStUDIntCommReinf'] = df.apply(interview_type, axis = 1)

In [65]:
def kStUDIntCommReinfDays (row):
    if row['kStUDIntCommReinf'] == 1:
        return row['COMMUNITY_REINFORCEMENT_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return     kStUDIntCommReinfDays (row)
    else:
        pd.NA     

df['kStUDIntCommReinfDays'] = df.apply(interview_type, axis = 1)

In [66]:
def kStUDIntCBT (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0:
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Cognitive Behavioral Therapy':
        return 1 
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return kStUDIntCBT (row)
    else:
        pd.NA    

df['kStUDIntCBT'] = df.apply(interview_type, axis = 1)

In [67]:
def kStUDIntCBTDays (row):
    if row['kStUDIntCBT'] == 1 :
        return row['COGNITIVE_BEHAVIORAL_THERAPY_NUMBER_OF_DAYS'] 
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return kStUDIntCBTDays (row)
    else:
        pd.NA    
    

df['kStUDIntCBTDays'] = df.apply(interview_type, axis = 1)

In [68]:
#Creating function to create column kStUDIntOther
def kStUDIntOther (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0:
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Other treatment approach':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return kStUDIntOther (row)
    else:
        pd.NA  

#Creating column kStUDIntOther via applying the functionkStUD_IntOther (x)
df['kStUDIntOther'] = df.apply(interview_type, axis = 1)

In [69]:
def kStUDIntOtherDays (row):
    if row['kStUDIntOther'] == 1:
        return row['OTHER_TREATMENT_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kStUDIntOtherDays (row)
    else:
        pd.NA     

df['kStUDIntOtherDays'] = df.apply(interview_type, axis = 1)

In [70]:
#Creating function to create column kStUDIntNotReceived
def kStUDIntNotReceived (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0:
        return -1 
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive any intervention for a stimulant use disorder [SKIP TO QUESTION 7]':
        return 1
    else:
        return 0

def interview_type(row):
    if row['InterviewType'] == 5:
        return  kStUDIntNotReceived (row)
    else:
        pd.NA      

#Creating column kStUDIntNotReceived via applying the function kStUD_IntNotReceived (x)
df['kStUDIntNotReceived'] = df.apply(interview_type, axis = 1)

In [71]:
def StUDIntAttended (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0:
        return -1
    elif row['kStUDIntContMgmt'] == 1:
        return 1
    elif row['kStUDIntCommReinf'] == 1:
        return 1
    elif row['kStUDIntCBT'] == 1:
        return 1
    elif row['kStUDIntOther'] == 1:
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  StUDIntAttended (row)
    else:
        pd.NA      

    
df['StUDIntAttended'] = df.apply( interview_type, axis = 1)

In [72]:
#Replacing null values with -9
df['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] = df['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'].fillna(-9)

#Creating function to create column PreviouslyDiagnosedTUD
def PreviouslyDiagnosedTUD (row):
    if row['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] == 'Yes':
        return 1
    elif row['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] == 'No [THE DISCHARGE INTERVIEW IS COMPLETE.]':
        return 0
    elif row['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] == -9:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  PreviouslyDiagnosedTUD (row)
    else:
        pd.NA 
    
#Creating column PreviouslyDiagnosedTUD via applying the function 
df['PreviouslyDiagnosedTUD'] = df.apply(interview_type, axis = 1)

In [73]:
# df['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] = df['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]

def kTUDMedNicotineRepl (row):
    if row['PreviouslyDiagnosedTUD'] == -9 or row['PreviouslyDiagnosedTUD'] == 0:
        return -1
    elif row['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Nicotine Replacement':
        return 1
    else:
        return 0  
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kTUDMedNicotineRepl (row)
    else:
        pd.NA 
        
    
df['kTUDMedNicotineRepl'] = df.apply(interview_type, axis =1 )

In [74]:
def kTUDMedNicotineReplDays (row):
    if row['kTUDMedNicotineRepl'] == 1:
        return row['NICOTINE_REPLACEMENT_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   kTUDMedNicotineReplDays (row)
    else:
        pd.NA     

df['kTUDMedNicotineReplDays'] = df.apply(interview_type, axis = 1)

In [75]:
def kTUDMedBupropion (row):
    if row['PreviouslyDiagnosedTUD'] == -9 or row['PreviouslyDiagnosedTUD'] == 0:
        return -1
    elif row['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Bupropion':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return  kTUDMedBupropion (row)
    else:
        pd.NA     

df['kTUDMedBupropion'] = df.apply(interview_type, axis = 1)

In [76]:
def kTUDMedBupropionDays (row):
    if row['kTUDMedBupropion'] == 1:
        return row['BUPROPION_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   kTUDMedBupropionDays (row)
    else:
        pd.NA
        
df['kTUDMedBupropionDays'] = df.apply(interview_type, axis = 1)

In [77]:
def kTUDMedVarenicline (row):
    if row['PreviouslyDiagnosedTUD'] == -9 or row['PreviouslyDiagnosedTUD'] == 0:
        return -1
    elif row ['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Varenicline':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   kTUDMedVarenicline (row)
    else:
        pd.NA
    
df['kTUDMedVarenicline'] = df.apply(interview_type, axis = 1)

In [78]:
def kTUDMedVareniclineDays (row):
    if row['kTUDMedVarenicline'] == 1:
        return row['VARENICLINE_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   kTUDMedVareniclineDays (row)
    else:
        pd.NA
df['kTUDMedVareniclineDays'] = df.apply(interview_type, axis = 1)

In [79]:
def kTUDMedNotReceived (row):
    if row['PreviouslyDiagnosedTUD'] == -9 or row['PreviouslyDiagnosedTUD'] == 0:
        return -1
    elif row['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive an FDA-approved medication for a tobacco use disorder [THE DISCHARGE INTERVIEW IS COMPLETE.]':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   kTUDMedNotReceived (row)
    else:
        pd.NA    

df['kTUDMedNotReceived'] = df.apply(interview_type, axis = 1)

In [80]:
def TUDMedTakenAsPrescribed (row):
    if row['PreviouslyDiagnosedTUD'] == -9 or row['PreviouslyDiagnosedTUD'] == 0:
        return -1
    elif row['kTUDMedNicotineRepl'] == 1:
        return 1
    elif row['kTUDMedBupropion'] == 1:
        return 1
    elif row['kTUDMedVarenicline'] == 1:
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return   TUDMedTakenAsPrescribed (row)
    else:
        pd.NA 
        
df['TUDMedTakenAsPrescribed'] = df.apply(interview_type, axis = 1)

In [81]:
#Creating the IntakeDate and InterviewCreateDate columns 
now = dt.now().strftime('%m/%d/%Y')

df['InterviewCreateDate'] = now

In [82]:
#Renaming column to create column SvcRecoveryPlanning
df.rename(columns = {'RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED_1':'SvcRecoveryPlanning'},inplace = True)

#Replacing null values with 0
df['SvcRecoveryPlanning'] = df['SvcRecoveryPlanning'].fillna(0)

def interview_type(row):
    if row['InterviewType'] == 5:
        return   row['SvcRecoveryPlanning']
    else:
        pd.NA 
    
df['SvcRecoveryPlanning'] = df.apply(interview_type, axis = 1)

In [83]:
#Inserting blank columns from section H

section_h_blank_col = [
'ReunitedChild',
'ReunitedChildWithAgcySupe',
'ReunitedChildWithoutAgcySupe',
'AvoidedOutOfHomePlacement',
'H1NoneOfAbove',
'HelpObPrivateHealthInsurance',
'HelpObMedicaid',
'HelpObMedicare',
'HelpObSSIOrSSDI',
'HelpObTANF',
'HelpObSNAP',
'HelpObOtherBenefit',
'HelpObOtherBenefitSpec',
'HelpObNoneOfTheAbove',
'EnrolledInSchool',
'EnrolledInSchoolHelp',
'EnrolledInVocTraining',
'EnrolledInVocTrainingHelp',
'CurrentlyEmployed',
'CurrentlyEmployedHelp',
'LivingInStableHousing',
'LivingInStableHousingHelp',
'MaintainFamilyResponsibilities',
'SupportToParentingAndRecovery',
'EnableFocusOnTreatment',
'SupportToParentingAndRecoveryH5',
'SexActivity',
'SexContacts',
'SexUnprot',
'SexUnprotectedHIV',
'SexUnprotectedInjDrugUser',
'SexUnprotectedHigh',
'PrEPOrTreat',
'HIVTestH',
'HIVTestFirstTime',
'HIVTestLocation',
'HIVTestReferralLocation',
'HIVTestReferralLocationSpec',
'HIVTestResultH',
'HIVTestConfirmatory',
'HIVTestConfirmatoryRes',
'TreatmentForHIVH',
'TreatmentForHIVLocation',
'TreatmentForHIVLocationSpec',
'HIVTestingPartners',
'HIVTestingPartnersNumber',
'HIVReferredPreEP',
'HIVReferredPEP',
'HIVReferredCounseling',
'HIVReferredNoMedications',
'HIVReferredNoCounseling',
'HCVTest',
'HCVTestConfirmatory',
'HCVTestResult',
'TreatmentForHCV',
'HBVTest',
'HBVTestResult',
'TreatmentForHBV',
'HepABVacc',
'HepABVaccReferral',
'PeerSupport',
'EnrolledInSchoolH8',
'EnrolledInSchoolHelpH8',
'EnrolledInVocTrainingH8',
'EnrolledInVocTrainingHelpH8',
'CurrentlyEmployedH8',
'CurrentlyEmployedHelpH8',
'LivingInStableHousingH8',
'LivingInStableHousingHelpH8',
'QualityOfLife',
'CommunicateWithProvider',
'ReduceSubstanceUse',
'ManageMentalHealthSymptoms',
'SupportRecovery',
'MentalHealthDisorderScreen',
'MentalHealthServiceReferred',
'MentalHealthServiceReceived',
'CoOccurringDisorderScreen',
'CoOccurringServiceReferred',
'CoOccurringServiceReceived',
'HelpedAvoidJusticeSystem'

]

df[section_h_blank_col] = df[section_h_blank_col] = ''

In [84]:
#Creating QAOnly column
df['QAOnly'] = ''

## Exporting GPRA 6

In [86]:
#Creating variable to export GPRA 6
GPRA6_SBIRT = df[['ClientID','FLWPClientContacted',	'FLWPStatus',	'FLWPStatusSpec',	'ReceivingServices',
                  'DischargeDate',	'DischargeStatusCompl',	'DischargeStatusTermReason',	'OtherDischargeStatTermRsnSpec',
                  'jHIVTest',	'jHIVTestResult',	'ODPreventionToolsProvided',	'VaccinatedStatusCOVID19',
                  'SvcCaseManagement',	'SvcIntensiveOutpatient',	'SvcInpatient',	'SvcOutpatient',	'SvcOutreach',
                  'SvcMethadone',	'SvcBuprenorphine',	'SvcNaltrexoneShortActing',	'SvcNaltrexoneLongActing',
                  'SvcDisulfiram',	'SvcAcamprosate',	'SvcNicotineReplacement',	'SvcBupropion',	'SvcVarenicline',
                  'SvcResidentialRehab',	'SvcHospitalInpatient',	'SvcFreeStandingRes',	'SvcAmbulatoryDetox',
                  'SvcAfterCare',	'SvcRecoverySupport',	'SvcOtherModalities',	'SvcOtherModalitesSpec',
                  'SvcScreening',	'SvcBriefIntervention',	'SvcBriefTreatment',	'SvcReferralTreatment',	
                  'SvcAssessment',	'SvcTreatmentPlanning',	'SvcRecoveryPlanning',	'SvcIndividualCouns',
                  'SvcGroupCouns',	'SvcContingencyManagement',	'SvcCommunityReinforcement',	'SvcCBT',
                  'SvcFamilyMarriageCouns',	'SvcCoOccurring',	'SvcPharmacological',	'SvcHIVAIDSCouns',
                  'SvcCulturalInterventions',	'SvcOtherClinicalCouns',	'SvcOtherClinicalCounsSpec',
                  'SvcFamilyServices',	'SvcChildCare',	'SvcPreEmployment',	'SvcEmploymentCoaching',	'SvcIndividualCoord',
                  'SvcTransportation',	'SvcHIVAIDSServices',	'SvcDrugFreeHousing',	'SvcHousingSupport',
                  'SvcHealthInsuranceEnrollment',	'SvcOtherCaseMgmt',	'SvcOtherCaseMgmtSpec',	'SvcMedicalCare',
                  'SvcAlcoholDrugTesting',	'SvcOBGYN',	'SvcHIVAIDSMedical',	'SvcHepatitisSupport',
                  'SvcOtherSTISupport',	'SvcDental','SvcOtherMedical','SvcOtherMedicalSpec',	'SvcContinuingCare',
                  'SvcRelapsePrevention',	'SvcRecoveryCoaching',	'SvcSelfHelpSupport',	'SvcSpiritualSupport',
                  'SvcOtherAfterCare',	'SvcOtherAfterCareSpec',	'SvcSubstanceAbuseEdu',	'SvcHIVAIDSEdu',
                  'SvcHepatitisEdu',	'SvcOtherSTIEdu',	'SvcNaloxoneTraining',	'SvcFentanylTestStripTraining',
                  'SvcOtherEdu',	'SvcOtherEduSpec',	'SvcPeerCoaching',	'SvcVocational',	'SvcRecoveryHousing',
                  'SvcRecoveryPlanningRSS',	'SvcRecoveryCaseManagement',	'SvcDrugFreeSocial',
                  'SvcInformationReferral',	'SvcOtherRecoverySupport',	'SvcOtherRecoverySupportSpec',
                  'SvcOtherPeerRecovery',	'SvcOtherPeerRecoverySpec',	'AttendedMostPlannedServices',
                  'ReceivedTelehealthServices',	'PreviouslyDiagnosedOUD',	'kOUDMedMethadone',	'kOUDMedMethadoneDays',	
                  'kOUDMedBuprenorphine',	'kOUDMedBuprenorphineDays',	'kOUDMedNaltrexone',	'kOUDMedNaltrexoneDays',
                  'kOUDMedXRNaltrexone','kOUDMedXRNaltrexoneDoses',	'kOUDMedNotReceived',	'OUDMedTakenAsPrescribed',
                  'PreviouslyDiagnosedAUD',	'kAUDMedNaltrexone',	'kAUDMedNaltrexoneDays',	'kAUDMedXRNaltrexone',
                  'kAUDMedXRNaltrexoneDoses',	'kAUDMedDisulfiram',	'kAUDMedDisulfiramDays',	'kAUDMedAcamprosate',
                  'kAUDMedAcamprosateDays',	'kAUDMedNotReceived',	'AUDMedTakenAsPrescribed',	'PreviouslyDiagnosedStUD',
                  'kStUDIntContMgmt',	'kStUDIntContMgmtDays',	'kStUDIntCommReinf',	'kStUDIntCommReinfDays',
                  'kStUDIntCBT',	'kStUDIntCBTDays',	'kStUDIntOther',	'kStUDIntOtherDays',	'kStUDIntNotReceived',
                  'StUDIntAttended',	'PreviouslyDiagnosedTUD',	'kTUDMedNicotineRepl',	'kTUDMedNicotineReplDays',
                  'kTUDMedBupropion',	'kTUDMedBupropionDays',	'kTUDMedVarenicline',	'kTUDMedVareniclineDays',
                  'kTUDMedNotReceived',	'TUDMedTakenAsPrescribed','ReunitedChild',	'ReunitedChildWithAgcySupe',
                  'ReunitedChildWithoutAgcySupe',	'AvoidedOutOfHomePlacement',	'H1NoneOfAbove',
                  'HelpObPrivateHealthInsurance',	'HelpObMedicaid',	'HelpObMedicare',	'HelpObSSIOrSSDI',
                  'HelpObTANF',	'HelpObSNAP',	'HelpObOtherBenefit',	'HelpObOtherBenefitSpec',	'HelpObNoneOfTheAbove',
                  'EnrolledInSchool',	'EnrolledInSchoolHelp',	'EnrolledInVocTraining',	'EnrolledInVocTrainingHelp',
                  'CurrentlyEmployed',	'CurrentlyEmployedHelp',	'LivingInStableHousing',	'LivingInStableHousingHelp',
                  'MaintainFamilyResponsibilities',	'SupportToParentingAndRecovery',	'EnableFocusOnTreatment',
                  'SupportToParentingAndRecoveryH5',	'SexActivity',	'SexContacts',	'SexUnprot',	
                  'SexUnprotectedHIV',	'SexUnprotectedInjDrugUser',	'SexUnprotectedHigh',	'PrEPOrTreat',
                  'HIVTestH',	'HIVTestFirstTime',	'HIVTestLocation',	'HIVTestReferralLocation',
                  'HIVTestReferralLocationSpec',	'HIVTestResultH',	'HIVTestConfirmatory',	
                  'HIVTestConfirmatoryRes',	'TreatmentForHIVH',	'TreatmentForHIVLocation',	'TreatmentForHIVLocationSpec',
                  'HIVTestingPartners',	'HIVTestingPartnersNumber',	'HIVReferredPreEP',	'HIVReferredPEP',	
                  'HIVReferredCounseling',	'HIVReferredNoMedications',	'HIVReferredNoCounseling',	'HCVTest',
                  'HCVTestConfirmatory',	'HCVTestResult',	'TreatmentForHCV',	'HBVTest',	'HBVTestResult',
                  'TreatmentForHBV',	'HepABVacc',	'HepABVaccReferral',	'PeerSupport',	'EnrolledInSchoolH8',
                  'EnrolledInSchoolHelpH8',	'EnrolledInVocTrainingH8',	'EnrolledInVocTrainingHelpH8',	'CurrentlyEmployedH8',
                  'CurrentlyEmployedHelpH8',	'LivingInStableHousingH8',	'LivingInStableHousingHelpH8',	'QualityOfLife',
                  'CommunicateWithProvider',	'ReduceSubstanceUse',	'ManageMentalHealthSymptoms',	'SupportRecovery',
                  'MentalHealthDisorderScreen',	'MentalHealthServiceReferred',	'MentalHealthServiceReceived',	'CoOccurringDisorderScreen',
                  'CoOccurringServiceReferred',	'CoOccurringServiceReceived',	'HelpedAvoidJusticeSystem','InterviewCreateDate','QAOnly']]


#Exporting GPRA 6
GPRA6_SBIRT.to_csv('GPRA6_SBIRT.csv', index = False, header = True, encoding = 'utf-8')

